In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import loadOSCdata
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

import psycopg2
import osgeo.ogr
import shapely
import shapely.wkt
%matplotlib inline

In [2]:
data = loadOSCdata.downloadData(OSCid=401441, X = True, Y = True, Z = True, output = 'csv', outputFile = 'data.csv')
data.head()

,timestamp,point_id,v_value,geometry,image_url,image_lab,trip_id
359,2017-05-30 10:27:18.191175,0.0,10.769757,POINT (-73.974177 40.6807485),http://storage4.openstreetcam.org/files/photo/...,acceptable,401441
464,2017-05-30 10:27:22.122909,1.0,10.747140,POINT (-73.97433599999999 40.68077),http://storage4.openstreetcam.org/files/photo/...,acceptable,401441
548,2017-05-30 10:27:25.199758,2.0,4.727365,POINT (-73.97447149999999 40.6807915),http://storage4.openstreetcam.org/files/photo/...,acceptable,401441
630,2017-05-30 10:27:28.167336,3.0,2.349433,POINT (-73.97454999999999 40.6807985),http://storage4.openstreetcam.org/files/photo/...,bad,401441
741,2017-05-30 10:27:32.227059,4.0,6.292053,POINT (-73.974553 40.680786),http://storage4.openstreetcam.org/files/photo/...,good,401441


In [3]:
connect_str = "dbname='squidbike' user='squidbike' \
host='rds-postgresql-10mintutorial.cbz1xmmdmpva.us-east-2.rds.amazonaws.com' \
port='5432' password='squidbikesql'"
conn = psycopg2.connect(connect_str)

In [4]:
cursor = conn.cursor()
cursor.execute("SELECT * from accelerometer2")
rows = cursor.fetchall()

In [ ]:
'''
link to aws PostGIS installation:

http://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/Appendix.PostgreSQL.CommonDBATasks.html#Appendix.PostgreSQL.CommonDBATasks.PostGIS

'''

In [7]:
cursor.execute("create extension postgis")

InternalError: current transaction is aborted, commands ignored until end of transaction block


In [5]:
#cursor.execute("DROP TABLE IF EXISTS bike_trip_testing")
cursor.execute("CREATE TABLE bike_trip_testing \
               (id SERIAL PRIMARY KEY,\
               timestamp Text,\
               point_id numeric(10,5),\
               v_value numeric(10,5),\
               geometry Geometry,\
               image_url Text,\
               image_lab Text,\
               trip_id Text\
               )")
cursor.execute("CREATE INDEX nycLionIndex ON bike_trip_testing USING GIST(Geometry)")

ProgrammingError: type "geometry" does not exist
LINE 1: ...    v_value numeric(10,5),               geometry Geometry, ...
                                                             ^


In [ ]:
#Saving records in the table
for index in range(len(data)):
    timestamp = data.ix[index, 'timestamp']
    point_id = data.ix[index, 'Street']
    v_value = data.ix[index, 'v_value']
    geometry = data.ix[index, 'geometry']
    image_url = data.ix[index, 'image_url']
    image_lab = data.ix[index, 'image_lab']
    trip_id = data.ix[index, 'trip_id']
    cursor.execute("INSERT INTO geoStreetTalk (timestamp, point_id, v_value, geometry, image_url, image_lab,\
    trip_id) \
    VALUES ('%s', '%d', '%d', ST_GeomFromText('%s', 4326), '%s',\
    '%s', '%s')"\
    %(timestamp, point_id, v_value, geometry, image_url, \
    image_lab, trip_id))
#     if index == 1000:
#         break
connection.commit()